In [ ]:
import numpy as np
from scipy.linalg import solve, schur
from scipy.integrate import solve_ivp
from scipy.sparse.linalg import eigs
from scipy.sparse.linalg import LinearOperator
from utility import orbit, BrusselatorModel
import sys
from scipy.sparse.linalg import eigs


In [ ]:
if __name__ == "__main__":
    param_file = "./brusselator_params_4.in"  # JSON file containing model parameters
    model = BrusselatorModel(param_file)
    print("Loaded parameters:", model.L)

f = model.dydt
Jacf = model.brusselator_jacobian


z_L = model.z_L
N = model.N
A, B = model.A, model.B
z = np.linspace(0, z_L, N)
perturb = np.sin(np.pi*(z/z_L))

X0 = A + 0.01*perturb
Y0 = B/A + 0.01*perturb

y0 = np.concatenate([X0[1:-1],Y0[1:-1]])

Max_iter = 100
epsilon = 1e-13
T_0 = model.T_ini

t_eval = np.linspace(0.0,20*T_0, 1000)

sol = solve_ivp(fun=f,t_span=[0.0, 20*T_0],
                t_eval=t_eval, 
                y0=y0, method='RK45', 
                **{"rtol": 1e-7,"atol":1e-9}
                )

y_T = sol.y[:,-1]


## Monodromy computation from the converged Newton method

In [ ]:
orbit_finder = orbit(f,y_T,T_0, Jacf,2, Max_iter, epsilon)

k, T_by_iter, y_by_iter, Norm_B, Norm_Deltay, monodromy_0, monodromy = orbit_finder.Newton_orbit(f,y_T,T_0, Jacf,2, Max_iter, epsilon)


In [ ]:
# phi, M = orbit_finder.integ_monodromy(y_T, T_0)

In [ ]:
np.iscomplexobj(M)

In [ ]:
T = T_by_iter[k-1]
y = np.array(y_by_iter[k-1])
rho = 1-0.25
eig, eigvec = np.linalg.eig(monodromy)
mask = np.abs(eig) - rho > 0.0
print(f"Number of Floquet multipliers outside the circle with radius {rho}\n",len(eig[mask]))

print("Spectral radius of the Monodromy matrix:\n",np.max(np.abs(eig)))

### Computing the dominant subspace using the Subsapce Iteration with projection algorithm
We initialize the subspace Vp with a member of the eigenvectors outside the unit cercle computed directly after the Newton method 

In [ ]:
p0, pe = 4,3
subspace_iter = 10

Ve_0 = np.real(eigvec[:p0+pe].T) # Taking the real part bc we want a real orthonormal basis of the dominant subspace of M
Ve, _ = np.linalg.qr(Ve_0)# Orthonormalization of Ve_0


In [ ]:
v0 = np.real(eigvec[0])

In [ ]:
eigen , Vp = orbit_finder.base_Vp(v0, y, T, f, Jacf, 6, epsilon)

In [ ]:
Re_sorted,Ye_sorted, Ve, We = orbit_finder.subsp_iter_projec(Ve, y, T, f, Jacf, p0, pe,subspace_iter, tol=1e-9)
eig_Re,_ = np.linalg.eig(Re_sorted)
mask1 = np.abs(eig_Re) - rho >  0.0
# mask = np.abs(eig) - rho > 0.0

In [ ]:
# mask1 = np.abs(eigen) - rho >  0.0

max_eigenvalues = np.sort(eig[mask])[::-1]    
print("Eigenvalues from subspace iteration \n",(eig_Re[mask1]))
print("The first p0 eigenvalues of the converged monodromy matrix \n", max_eigenvalues)
# Ve, _ = np.linalg.qr(Ve, mode='complete')
# np.dot(Ve.H, Ve)
#  - np.eye(len(y0))
# Ve = np.matrix(Ve)
np.allclose(np.eye(p0+pe),np.dot(Ve.T, Ve))

# np.iscomplexobj(Ve)

In [ ]:
mask1

In [ ]:
import numpy as np
rng = np.random.default_rng()
a = rng.normal(size=(9, 6))
Q, R = np.linalg.qr(a)
np.dot(Q.T, Q)  # a does equal QR
# R2 = np.linalg.qr(a, mode='r')
# np.allclose(R, R2)  # mode='r' returns the same R as mode='full'
# a = np.random.normal(size=(3, 2, 2)) # Stack of 2 x 2 matrices as input
# Q, R = np.linalg.qr(a)
# Q.shape
# R.shape
# np.allclose(a, np.matmul(Q, R))
(Q.T@Q).shape

In [ ]:
eigenVp, Vp = orbit_finder.base_Vp(v0=v0, y0=y0,T=T, f=f,Jacf=Jacf,p=p,epsilon=1e-6)


In [ ]:
eig[mask]

In [ ]:
np.abs(eigenVp)

In [ ]:
np.linalg.norm(eig[mask] - eigenVp)